# Minigame 13: Choose one area to refine

This is essentially the global environment for refinement where the action is choosing one geometric area to refine.

In [1]:
import math
from math import cos,sin
import random

In [2]:
import gym
from gym import spaces, utils
import numpy as np
import ray
import ray.rllib.agents.ppo as ppo
import ray.rllib.agents.dqn as dqn
from ray.tune.logger import pretty_print
from os.path import expanduser, join
import os

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
from glvis import glvis, to_stream

In [4]:
from mfem import path
import mfem.ser as mfem

Define some synthetic test functions: steps and bumps.

In [5]:
def bump(x):
    rsq = x[0]**2 +x[1]**2
    return math.exp(-rsq)

In [6]:
def smooth_step(x):
    return 0.5*(1.0 +math.tanh(x[0]))

Create classes where we can set the parameters and then eval a bunch of points.

In [7]:
class Bump(mfem.PyCoefficient):
    
    def SetParams(self):
        self.width = random.uniform(0.05,0.5)
        self.xc = [0.5,0.5]
        self.dx = [random.uniform(-0.5, 0.5),random.uniform(-0.5, 0.5)]
        y1 = random.uniform(0.1,0.9)
        y2 = random.uniform(0.1,0.9)
        self.floor = min(y1,y2)
        self.ceiling = max(y1,y2)
        self.height = self.ceiling -self.floor
        
    def Print(self):
        print("width = %f" % self.width)
        print("xc = %f,%f" % (self.xc[0],self.xc[1]))
        print("dx = %f,%f" % (self.dx[0],self.dx[1]))
        print("floor = %f" % self.floor)
        print("ceil = %f"  % self.ceiling)
        print("height = %f"% self.height)
        
    def EvalValue(self, x):
        v = self.floor +self.height*bump((x-self.xc+self.dx)/self.width)
        #assert v > 0.0
        #assert v < 1.0
        return v

In [8]:
class DynamicBump(mfem.PyCoefficient):
    
    def SetParams(self):
        self.x0 = 0.5
        self.y0 = 0.5
        self.u = 1.0
        self.width = 0.1
        self.t = 0.0
        
    def SetTime(self,t):
        self.t = t
        
    def EvalValue(self,x):
        self.xc = [self.x0 +self.u*self.t, self.y0]
        return bump((x-self.xc)/self.width)

In [9]:
class DiscreteBump(mfem.PyCoefficient):
    
    def SetParams(self):
        nlocs = 5
        nmesh = 5
        i = random.randrange(nlocs)
        j = random.randrange(nlocs)
        dx = 1.0/nlocs
        dx_mesh = 1.0/nmesh
        self.xc = [0.,0.]
        self.xc[0] = i*dx+0.5*dx
        self.xc[1] = j*dx+0.5*dx
        self.width = dx_mesh/2
        #print("(%d,%d)" % (i,j))
        #print("(%f,%f)" % (self.xc[0],self.xc[1]))
        
    def Print(self):
        pass
        
    def EvalValue(self, x):
        v = bump((x-self.xc)/self.width)
        assert v >= 0.0
        assert v <= 1.0
        return v

Visualize an instance of the test function. Note that each instance has randomly chosen parameters.  For the steps, it's a rotation angle and a displacement.  For the bumps, it's a width and a displacement.

In [10]:
mesh1 = mfem.Mesh('inline-quad.mesh')
mesh1.UniformRefinement()
mesh1.UniformRefinement()
fec1 = mfem.L2_FECollection(p=1, dim=2)
fes1 = mfem.FiniteElementSpace(mesh1, fec1)
u1 = mfem.GridFunction(fes1)
c1 = Bump()
c1.SetTime(0.0)
c1.SetParams()
u1.ProjectCoefficient(c1)

In [11]:
ctr = mfem.Vector(2)
mesh1.GetElementCenter(1,ctr)
ctr.GetDataArray()

array([0.0375, 0.0125])

In [12]:
glvis(to_stream(mesh1,u1)+'keys Rjlmc',500,500)

glvis()

Create the gym environment.

In [13]:
class RegionGame(gym.Env):
    
    class u0_coeff(mfem.PyCoefficient):
        
        def SetParams(self):
            self.fn = Bump()
            self.fn.SetParams()
        
        def SetTime(self,t):
            self.fn.SetTime(t)
            
        def EvalValue(self, x):
            return self.fn.EvalValue(x)
        
    # In RLlib, you need the config arg
    def __init__(self,config):
        self.t = 0.0
        self.dt = 0.5
       
        self.meshfile = 'inline-quad-5.mesh'
        
        # keep a copy of the unrefined mesh so we can restore it
        self.mesh0 = mfem.Mesh(self.meshfile)
        self.mesh = mfem.Mesh(self.meshfile)
        
        # The only reason we need to create a fespace and gf here
        # is to find the sizes needed for the action and observation spaces
        dim = self.mesh.Dimension()
        self.order = 1
        self.fec = mfem.L2_FECollection(self.order, dim)
        self.fes = mfem.FiniteElementSpace(self.mesh, self.fec)
        self.u = mfem.GridFunction(self.fes);

        # actions are: refine one circle with (x,y,r)
        self.action_space = spaces.Box( np.array([0.,0.,0.]), np.array([1.0,1.0,1.0]))
        
        # observation space is a grid of points. To avoid sampling element boundaries,
        # sample from centers of equispaced grid which is a multiple of the element
        # grid.
        self.obsx = 42
        self.obsy = 42

        self.observation_space = spaces.Box(-1.0, 2.0, shape=(self.obsx,self.obsy,1))
        
        self.get_obs_points()
        # call reset to create the first synthetic function
        self.reset()
        
        #self.gl = GlvisWidget(get_solnstream(self.mesh,self.u))
        
    def get_ne(self):
        return self.mesh.GetNE()
    
    def get_size(self):
        return self.u.Size()
    
    # Compute L2 error wrt to the analytic fn definition
    def get_error(self):
        err = self.u.ComputeL2Error(self.u0)
        return err

    # Manually refine the elements in the array elems
    def refine_elems(self, elems):
        self.mesh.GeneralRefinement(mfem.intArray(elems))
        self.fes.Update()
        self.u.Update()
        
    def no_refinement_error(self,t):
        del self.mesh
        self.mesh = mfem.Mesh(self.mesh0)

        del self.fes
        self.fes = mfem.FiniteElementSpace(self.mesh, self.fec)

        del self.u
        self.u = mfem.GridFunction(self.fes)
        self.u0.SetTime(t)
        self.u.ProjectCoefficient(self.u0)     
        return self.u.ComputeL2Error(self.u0)

    # precompute the observation points and the elements and integration points we need
    def get_obs_points(self):
        n = math.sqrt(self.mesh.GetNE())
        dx = 1.0/self.obsx
        dy = 1.0/self.obsy
        self.sample_els = []
        self.sample_ips = []
        for j in range(self.obsy):
            for i in range(self.obsx):
                pt = [i*dx+0.5*dx,j*dy+0.5*dy]
                n, el, ip = self.mesh.FindPoints([pt])
                #assert n == 1
                #assert ip[0].x > 0.0
                #assert ip[0].x < 1.0
                #assert ip[0].y > 0.0
                #assert ip[0].y < 1.0
                #assert el[0] >= 0
                #assert el[0] < self.mesh.GetNE()
                # copy these so they won't be destroyed when mesh goes away?
                ip0 = mfem.IntegrationPoint()
                ip0.x = ip[0].x
                ip0.y = ip[0].y
                self.sample_els.append(el[0])
                self.sample_ips.append(ip0)
         
    def get_obs(self):
        state = np.empty((self.obsx,self.obsy,1))
        k = 0
        for j in range(self.obsy):
            for i in range(self.obsx):
                #assert k < len(self.sample_els)
                #assert k < len(self.sample_ips)
                el = self.sample_els[k]
                #assert el >= 0
                #assert el < self.mesh.GetNE()
                ip = self.sample_ips[k]
                #assert ip.x >= 0.0, "k={}, i={}, j={}, x is {}".format(k,i,j,ip.x)
                #assert ip.x <= 1.0, "k={}, i={}, j={}, x is {}".format(k,i,j,ip.x)
                #assert ip.y >= 0.0, "k={}, i={}, j={}, y is {}".format(k,i,j,ip.y)
                #assert ip.y <= 1.0, "k={}, i={}, j={}, y is {}".format(k,i,j,ip.y)
                v = self.u.GetValue(self.sample_els[k],self.sample_ips[k])
                state[i][j] = v
                if (v > 2.0 or v < -1.0):
                    print("element %d" % self.sample_els[k])
                    print("ip.x = %f" % self.sample_ips[k].x)
                    print("ip.y = %f" % self.sample_ips[k].y)
                    print("%d,%d -> %f" % (i,j,v))
                    print("%d,%d -> %f" % (i,j,state[i][j]))
                    self.u0.Print()
                k += 1
        self.state = state
        return state
            
    # action is the number of the element to refine
    def step(self, action):
        
        # store orig state
        state = self.get_obs()
        
        # get action params
        xc = action[0]
        yc = action[1]
        rc = action[2]
        
        self.t += self.dt
        
        # The reward is the error reduction per dof relative to no refinement
        # reward = error_no_refinement/old_dofs -error_refinement/new_dofs
        error_no_refine = self.get_error()
        dofs_no_refine = self.u.Size()
        
        #print("error no refine: %e" % error_no_refine)
        
        # find the element centers intersected by the circle
        ctr = mfem.Vector(2)
        elems = []
        for k in range(self.mesh.GetNE()):
            self.mesh.GetElementCenter(k,ctr)
            dx = ctr[0]-xc
            dy = ctr[1]-yc
            dist = math.sqrt(dx**2 +dy**2)
            if (dist < rc):
                elems.append(k)
                
        # refine the elements and re-project
        self.refine_elems(elems)
        self.u.ProjectCoefficient(self.u0)
        
        error_refined = self.get_error()
        dofs_refined = self.u.Size()
        
        ratio_no_refine = 1.0/(error_no_refine*dofs_no_refine)
        ratio_refined = 1.0/(error_refined*dofs_refined)
        
        # the reward is the error reduction per dof
        # relative to no refinement
        # reward = ratio_refined -ratio_no_refine
        
        # reward is error reduction, but only down to a threshold
        #thresh = 1.e-4
        newdofs = dofs_refined -dofs_no_refine
        profit = error_no_refine-error_refined
        
        #if error_no_refine <= thresh:
        #    reward = -newdofs
        #else:
        #    reward = profit
        #    if newdofs != 0:
        #        reward /= newdofs
        if newdofs > 0:
            profit /= newdofs
            
        reward = profit*1.e6
            
        #print("error_no_refine = %e" % error_no_refine)
        #print("error_refined = %e" % error_refined)
        #print("profit = %e" % profit)
        #print("newdofs = %d" % newdofs)
        #print("newdofs = %d" % newdofs)
        #print("reward = %e" % reward)

        # update observation
        #state = self.get_obs()
        
        done = False
        if (self.t >= 1.0):
            done = True
        
        done = True
        return state, reward, done, {}

    
    # similar to reset, but do not choose a new function
    def reinit(self):
        del self.mesh
        self.mesh = mfem.Mesh(self.mesh0)

        del self.fes
        self.fes = mfem.FiniteElementSpace(self.mesh, self.fec)

        del self.u
        self.u = mfem.GridFunction(self.fes)
        self.u.ProjectCoefficient(self.u0)
                
        return self.get_obs()
    
    # every reset of the env chooses a new synthetic function
    def reset(self):
        self.u0 = self.u0_coeff()
        self.u0.SetParams()
        self.t = 0
        self.u0.SetTime(self.t)
        return self.reinit()
    
    def render(self):
        return glvis(to_stream(self.mesh,self.u) + 'keys Rjlmc',600,600)

Instantiate the environment and sanity check it.

In [14]:
env = RegionGame(None)
env.render()

/home/rwa/pyvenv/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


glvis()

In [15]:
env.step([0.5,0.5,0.22])
env.render()

glvis()

def find_optimal(obs):
    u0 = mfem.Vector(obs)
    maxr = 0.0;
    maxel = -1;
    env.reinit()
    ne = env.get_ne()
    for n in range(ne):
        env.reinit()
        state, reward, done, info = env.step(n)
        if reward > maxr:
            maxr = reward
            maxel = n
    #print("max reward is %f by refining element %d" % (maxr, maxel))
    env.reinit()
    return maxel, maxr

def find_dgjumps(env):
    
    # put the L2 gridfunction into a coefficient so we can project it
    u_disc_coeff = mfem.GridFunctionCoefficient(env.u)
    h1_fec = mfem.H1_FECollection(p=1, dim=2)
    h1_fes = mfem.FiniteElementSpace(env.mesh, h1_fec)
    u_h1 = mfem.GridFunction(h1_fes)
    u_h1.ProjectDiscCoefficient(u_disc_coeff, mfem.GridFunction.ARITHMETIC)
    
    # put the H1 smoothed function into a coefficient
    u_h1_coeff = mfem.GridFunctionCoefficient(u_h1)
    
    # create a 0-order L2 field to hold errors
    l2_0_fec = mfem.L2_FECollection(p=0,dim=2)
    l2_0_fes = mfem.FiniteElementSpace(env.mesh,l2_0_fec)

    # Compute elementwise "errors" between continuous and discontinuous fields
    err_gf = mfem.GridFunction(l2_0_fes);
    env.u.ComputeElementL2Errors(u_h1_coeff, err_gf);
    
    best_action = np.argmax(err_gf.GetDataArray())
    
    state, reward, done, info = env.step(best_action)
    #env.reinit()
    
    return best_action, reward

Ok, try training a policy:

In [16]:
ray.shutdown()
ray.init(ignore_reinit_error=True)
#config = dqn.DEFAULT_CONFIG.copy()
#config['num_workers'] = 3
#config['train_batch_size'] = 4000 # raising this from default 32 accelerates training substantially
#config['lr'] = 0.001

config = ppo.DEFAULT_CONFIG.copy()
config['train_batch_size'] = 2000
config['num_workers'] = 3
#config['framework'] = 'tfe'
agent = ppo.PPOTrainer(config, env=RegionGame)
config

2021-02-26 18:53:08,740	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265
2021-02-26 18:53:11,091	INFO trainer.py:591 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-02-26 18:53:11,091	INFO trainer.py:618 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=3092) WARNING:tensorflow:From /home/rwa/pyvenv/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=3092) Instructions for updating:
(pid=3092) non-resource variables are not supported in the long term
(pid=3091) WARNING:tensorflow:From /home/rwa/pyvenv/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=3091) Ins

{'num_workers': 3,
 'num_envs_per_worker': 1,
 'create_env_on_driver': False,
 'rollout_fragment_length': 200,
 'batch_mode': 'truncate_episodes',
 'num_gpus': 0,
 'train_batch_size': 2000,
 'model': {'fcnet_hiddens': [256, 256],
  'fcnet_activation': 'tanh',
  'conv_filters': None,
  'conv_activation': 'relu',
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': True,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action': False,
  'lstm_use_prev_reward': False,
  '_time_major': False,
  'framestack': True,
  'dim': 84,
  'grayscale': False,
  'zero_mean': True,
  'custom_model': None,
  'custom_model_config': {},
  'custom_action_dist': None,
  'custom_preprocessor': None,
  'lstm_use_prev_action_reward': -1},
 'optimizer': {},
 'gamma': 0.99,
 'horizon': None,
 'soft_horizon': False,
 'no_done_at_end': False,
 'env_config': {},
 'env': None,
 'normalize_actions': False,
 'clip_rewards': None,
 'clip_actions': True,
 'preproce

%%time
for n in range(1):
    result = agent.train()
    print("episode reward mean: %f " % result["episode_reward_mean"])

Create a convenience function for applying a policy to a given observation

In [17]:
def apply_policy(model, obs):
    for k in range(len(obs)):
        val = obs[k]
    action = agent.compute_action(obs, explore=False) # use deterministic mode
    state, reward, done, info = env.step(action)
    #print("policy chooses action %d with reward %f" % (action, reward))
    return action, reward

Brute force search for the best choice by trying each one, remembering to reset the environment after each action and after we're done.

obs = env.reset()
maxel, maxr = find_optimal(obs)
env.refine_elems([maxel])
env.render()

env.reinit()
action, reward = find_dgjumps(env)
env.render()

Run a more systematic evaluation using an ensemble of samples:

def eval_ensemble(model, ntrials):
    ncorrect = 0.0
    sumsq = 0.0
    maxerrsq = 0.0
    dg_ncorrect = 0.0
    dg_sumsq = 0.0
    dg_maxerrsq = 0.0
    for n in range(ntrials):
        obs = env.reset()
        bestaction, bestreward = find_optimal(obs)
        obs = env.reinit()
        dgaction, dgreward = find_dgjumps(env)
        obs = env.reinit()
        action, reward = apply_policy(model,obs)
        err = bestreward-reward
        maxerrsq = max(err*err,maxerrsq)
        sumsq += err*err
        dg_err = bestreward-dgreward
        dg_maxerrsq = max(dg_err*dg_err,dg_maxerrsq)
        dg_sumsq += dg_err*dg_err
        if (bestaction == action):
            ncorrect += 1
        if (bestaction == dgaction):
            dg_ncorrect += 1
    rms = math.sqrt(sumsq/ntrials)
    corr = 100.*ncorrect/ntrials
    print("policy rms error: ",rms,flush=True)
    print("policy max sq error: ",maxerrsq,flush=True)
    print("policy % correct: ",corr,flush=True)
    dg_rms = math.sqrt(dg_sumsq/ntrials)
    dg_corr = 100.*dg_ncorrect/ntrials
    print("dg rms error: ",dg_rms,flush=True)
    print("dg max sq error: ",dg_maxerrsq,flush=True)
    print("dg % correct: ",dg_corr,flush=True)
    return rms, math.sqrt(maxerrsq), corr, dg_rms, math.sqrt(dg_maxerrsq), dg_corr

eval_ensemble(model, 100)

Run a few eval sample sizes to get a sense of how many are needed to estimate the metrics of the policy

eval_ensemble(model, 200)

eval_ensemble(model, 400)

Let's see if the training process is making progress:

In [18]:
total_episodes = 1.e6
#nbatches = int(total_episodes/config['timesteps_per_iteration'])
nbatches = int(total_episodes/config['train_batch_size'])

eval_period = 50
neval = 0

checkpoint_period = 25

#config['train_batch_size'] = int(batch_size)
#agent = ppo.PPOTrainer(config, env=RegionGame)
#agent = dqn.DQNTrainer(config, env=RegionGame)
#policy = agent.get_policy()
#model = policy.model

rms = [0.0] * nbatches
cor = [0.0] * nbatches
maxerr = [0.0] * nbatches

dg_rms = [0.0] * nbatches
dg_cor = [0.0] * nbatches
dg_maxerr = [0.0] * nbatches

for n in range(nbatches):
    print("iteration %d/%d of %d" % (n,nbatches,config['train_batch_size']))
    if ( n and n % checkpoint_period == 0):
        checkpoint_path = agent.save()
        print(checkpoint_path)
    if ( n and eval_period and n % eval_period == 0):
        print("eval...")
        rms[n], maxerr[n], cor[n], dg_rms[n], dg_maxerr[n], dg_cor[n] = eval_ensemble(model, neval)
    result = agent.train()


(pid=3092) 2021-02-26 18:53:17,253	WARNING deprecation.py:30 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!


iteration 0/500 of 2000
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


(pid=3092) WARNING:tensorflow:From /home/rwa/pyvenv/lib/python3.6/site-packages/ray/rllib/policy/tf_policy.py:852: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=3092) Instructions for updating:
(pid=3092) Prefer Variable.assign which has equivalent behavior in 2.X.
(pid=3091) WARNING:tensorflow:From /home/rwa/pyvenv/lib/python3.6/site-packages/ray/rllib/policy/tf_policy.py:852: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=3091) Instructions for updating:
(pid=3091) Prefer Variable.assign which has equivalent behavior in 2.X.
(pid=3090) WARNING:tensorflow:From /home/rwa/pyvenv/lib/python3.6/site-packages/ray/rllib/policy/tf_policy.py:852: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=3090) Instructions for updating:
(pid=3090) Prefer Variable.assign which has equivalent behavior in 2.X.


iteration 1/500 of 2000
iteration 2/500 of 2000
iteration 3/500 of 2000
iteration 4/500 of 2000
iteration 5/500 of 2000
iteration 6/500 of 2000
iteration 7/500 of 2000
iteration 8/500 of 2000
iteration 9/500 of 2000
iteration 10/500 of 2000
iteration 11/500 of 2000
iteration 12/500 of 2000
iteration 13/500 of 2000
iteration 14/500 of 2000
iteration 15/500 of 2000
iteration 16/500 of 2000
iteration 17/500 of 2000
iteration 18/500 of 2000
iteration 19/500 of 2000
iteration 20/500 of 2000
iteration 21/500 of 2000
iteration 22/500 of 2000
iteration 23/500 of 2000
iteration 24/500 of 2000
iteration 25/500 of 2000
/home/rwa/ray_results/PPO_RegionGame_2021-02-26_18-53-113z9prstd/checkpoint_25/checkpoint-25
iteration 26/500 of 2000
iteration 27/500 of 2000
iteration 28/500 of 2000
iteration 29/500 of 2000
iteration 30/500 of 2000
iteration 31/500 of 2000


KeyboardInterrupt: 

In [34]:
obs = env.reset()
apply_policy(agent.get_policy().model, obs)
env.render()

glvis()

In [ ]:
%matplotlib inline
isteps = list(range(nbatches))
asteps = [i*config['train_batch_size'] for i in isteps]
import matplotlib.pyplot as plt
ax = plt.subplot(211)
ax.set_ylim(0.00001,0.01)
ax.set_ylabel('Error')
line1, = plt.semilogy(asteps,rms[:nbatches], marker='o')
line2, = plt.semilogy(asteps,dg_rms[:nbatches], marker='x')
line3, = plt.semilogy(asteps,maxerr[:nbatches], marker='.')
line4, = plt.semilogy(asteps,dg_maxerr[:nbatches], marker='+')

line1.set_label('RL rms')
line2.set_label('DG rms')
line3.set_label('RL max')
line4.set_label('DG max')
ax.legend()
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

ax = plt.subplot(212)
ax.set_ylim(0,100)
ax.set_ylabel('% correct')
ax.set_xlabel('training episodes')
line1, = plt.plot(asteps,cor[:nbatches], marker='o')
line2, = plt.plot(asteps,dg_cor[:nbatches], marker='x')
line1.set_label('RL policy')
line2.set_label('DG')
ax.legend()
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))